<a href="https://colab.research.google.com/github/SijieQiu/RQ/blob/main/812.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving noname.json to noname.json


In [ ]:
!pip install tomotopy
import tomotopy as tp
import json

# Load the JSON file
with open('noname.json', 'r') as f:
    data = json.load(f)
# 自定义停用词
stop_words = set(["around", "Sandy", "Nish", "Stuart", "him.", "Hector", "Shazia", "Cole", "Nicola", "Krish", "Bing", "in.", "them.", "her.", "Parke", "also", "already", "onto", "up.", "is.", "it.", "out.", "Iain", "Dr.", "it,"])


# Assuming 'data' is a list of documents
training_data = data  # Adjust this if the JSON structure is different

# Number of topics to return
num_topics = 60
# Number of topic words to print out
num_topic_words = 10

# Initialize the model
model = tp.LDAModel(k=num_topics)

# Add each document to the model, after splitting it up into words
for document in training_data:
    text = document.get('content', '') # Extract the text content from the dictionary, handle cases where 'content' might be missing
    model.add_doc(text.strip().split()) # Now apply strip and split to the extracted text string

print("Topic Model Training...\n\n")
# Iterate over the data 10 times
iterations = 10
for i in range(0, 100, iterations):
    model.train(iterations)
    print(f'Iteration: {i}\tLog-likelihood: {model.ll_per_word}')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 100.0 MB/s eta 0:00:00
Topic Model Training...


Iteration: 0	Log-likelihood: -9.96261410525256
Iteration: 10	Log-likelihood: -9.727460009097932
Iteration: 20	Log-likelihood: -9.581644375113264
Iteration: 30	Log-likelihood: -9.488013059105336
Iteration: 40	Log-likelihood: -9.442150056277033
Iteration: 50	Log-likelihood: -9.40929055923017
Iteration: 60	Log-likelihood: -9.369108565218408
Iteration: 70	Log-likelihood: -9.337682632240996
Iteration: 80	Log-likelihood: -9.306193707548902
Iteration: 90	Log-likelihood: -9.284995381628084


**重新清理数据**

In [ ]:
!pip install tomotopy
import tomotopy as tp
import json

# 自定义停用词
stop_words = set(["around", "Sandy", "Nish", "Stuart", "him.", "Hector", "Shazia", "Cole", "Nicola", "Krish", "Bing", "in.", "home,", "that.", "them.", "leave,", "herself.", "that,", "back.", "work,", "leave.", "head.", "life.", "gone,", "her.", "Parke", "also", "already", "onto", "up.", "is.", "it.", "out.", "Iain", "Dr.", "it,", "them,", "day,", "room,", "up,", "him,", "house,", "car,", "later,", "out,", "him,", "her,", "time.", "house.", "down.", "together.", "show.", "station.", "one.", "leaves.", "off.", "work.", "again.", "do.", "bed.", "it'll", "(),", "day.", "away.", "there.", "home.", "room.", "car.", "on.", "door.", "go."])

file_name = 'noname.json'

# 读取JSON文件
with open(file_name, 'r') as input_file:
    data = json.load(input_file)

# 确定要创建的主题数量
num_topics = 63

# 创建LDA模型
mdl = tp.LDAModel(k=num_topics, min_cf=10, rm_top=5)

# 添加文档到模型中
for index, row in enumerate(data):
    if index % 1000 == 0 and index > 0:
        print('Adding document #%s' % index)
    content = row['content']
    # 分词并过滤停用词
    clean_document = [token for token in content.strip().split() if len(token) > 2 and token not in stop_words]
    mdl.add_doc(clean_document)

# 训练模型
print('Starting training model')
for i in range(0, 100, 10):
    mdl.train(10)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, mdl.ll_per_word))

# 打印每个主题的前10个单词
for k in range(mdl.k):
    print('Top 10 words of global topic #{}: {}'.format(k, mdl.get_topic_words(k, top_n=10)))

    # 应用一致性评估
print("\nCalculating topic coherence...")
coh = tp.coherence.Coherence(mdl, coherence='c_v')
average_coherence = coh.get_score()
coherence_per_topic = [coh.get_score(topic_id=k) for k in range(mdl.k)]

print('==== Coherence : {} ===='.format('c_v'))
print('Average:', average_coherence, '\nPer Topic:', coherence_per_topic)

Starting training model
Iteration: 0	Log-likelihood: -6.632333380857742
Iteration: 10	Log-likelihood: -6.498546179151325
Iteration: 20	Log-likelihood: -6.477008511176746
Iteration: 30	Log-likelihood: -6.436472588566772
Iteration: 40	Log-likelihood: -6.352307928927788
Iteration: 50	Log-likelihood: -6.395939377286426
Iteration: 60	Log-likelihood: -6.386764260875732
Iteration: 70	Log-likelihood: -6.387000492277983
Iteration: 80	Log-likelihood: -6.325222256723427
Iteration: 90	Log-likelihood: -6.335757492885692
Top 10 words of global topic #0: [('room', 0.18893004953861237), ('back', 0.14534758031368256), ('notices', 0.08723759651184082), ('hear', 0.08723759651184082), ('camera', 0.08723759651184082), ('night', 0.08723759651184082), ('let', 0.05818261206150055), ('walks', 0.05818261206150055), ('since', 0.05091886594891548), ('falls', 0.036391373723745346)]
Top 10 words of global topic #1: [('takes', 0.20958755910396576), ('still', 0.1687023937702179), ('body', 0.12270659953355789), ('thin

In [ ]:
# 假设 `mdl` 是已经训练好的 LDA 模型，且 `num_topics` 已定义

topics = []
topic_individual_words = []
num_topic_words = 10  # 定义每个主题打印的单词数量

for topic_number in range(0, num_topics):
    # 获取指定主题的前 num_topic_words 个单词及其概率
    topic_words = ' '.join(word for word, prob in mdl.get_topic_words(topic_id=topic_number, top_n=num_topic_words))
    # 将这些单词作为一个字符串加入到 topics 列表中
    topics.append(topic_words)
    # 将这些单词分割成单个单词列表，并加入到 topic_individual_words 列表中
    topic_individual_words.append(topic_words.split())
    # 打印出每个主题及其对应的单词
    print(f"✨Topic {topic_number}✨\n\n{topic_words}\n")


✨Topic 0✨

room back notices hear camera night let walks since falls

✨Topic 1✨

takes still body think plays trying introduces leaves took want

✨Topic 2✨

thinks runs something happened stop police looks checks start talking

✨Topic 3✨

just know calls wants way making likes saw tell old

✨Topic 4✨

first story dead stop used eventually made case agrees party

✨Topic 5✨

sex actually months party away questions nothing house everything say

✨Topic 6✨

see will never work watching going digital saying introduces reveals

✨Topic 7✨

goes going watch seen now life feels thinks looking hand

✨Topic 8✨

another must glass two taking reveals might said bed part

✨Topic 9✨

real reveals house social know bed talking part begs face

✨Topic 10✨

telling everything everyone stops house tries just real working see

✨Topic 11✨

realizes follows much leads seen together come took happened check

✨Topic 12✨

years behind watches suddenly finds drives knife monkey leaving hit

✨Topic 13✨

even use 

num_topics = 10

Average: 0.41804842393845326

Per Topic: [0.5169136554002762, 0.3560481034219265, 0.41875907927751543, 0.4143201932311058, 0.40790387988090515, 0.4395752564072609, 0.356370298191905, 0.5073668658733368, 0.4230355493724346, 0.3401913583278656]

num_topics = 20

Average: 0.49108936678618187

Per Topic: [0.5248529732227325, 0.46410454511642457, 0.4734464451670647, 0.5030207961797715, 0.47880403995513915, 0.373695570230484, 0.4416766300797462, 0.49428573697805406, 0.46930399537086487, 0.5265482693910599, 0.5763946920633316, 0.49022464752197265, 0.5073260366916656, 0.5322769790887832, 0.469846174120903, 0.4155075691640377, 0.45041089206933976, 0.5408616840839386, 0.5519600629806518, 0.537239596247673]

num_topics = 60

Average: 0.48340031230511754

Per Topic: [0.559427335858345, 0.4700973004102707, 0.45365156084299085, 0.5551360338926316, 0.43146497681736945, 0.515483433008194, 0.4861409291625023, 0.4700476065278053, 0.5128947779536247, 0.47375204414129257, 0.4205316722393036, 0.4488257527351379, 0.4650417402386665, 0.45374039113521575, 0.5242934808135032, 0.43542255461215973, 0.5158875763416291, 0.46459038853645324, 0.45116778314113615, 0.44687093496322633, 0.44306353721767666, 0.46871467083692553, 0.4391462944447994, 0.5008008301258087, 0.49071610569953916, 0.4913387268781662, 0.45355403125286103, 0.4284819602966309, 0.46973030269145966, 0.5210075795650482, 0.45926999151706693, 0.4667395204305649, 0.4399174004793167, 0.505402797460556, 0.5392451345920563, 0.6103819400072098, 0.4520705632865429, 0.4410939395427704, 0.5401126950979233, 0.49913442730903623, 0.4656765475869179, 0.4731669291853905, 0.4137230269610882, 0.4979417771100998, 0.5206837505102158, 0.4645103797316551, 0.4873234137892723, 0.4951589718461037, 0.5633009061217308, 0.5157281428575515, 0.4766166269779205, 0.47536561712622644, 0.5305597513914109, 0.497916379570961, 0.49743774235248567, 0.5164845377206803, 0.46330140084028243, 0.4956053584814072, 0.47622239738702776, 0.4629043586552143]

num_topics = 70

Average: 0.4826403980728771

Per Topic: [0.46928040981292723, 0.503042659163475, 0.5047200702130794, 0.4758301451802254, 0.4700105607509613, 0.4928063049912453, 0.42060745432972907, 0.44676233679056165, 0.44551299810409545, 0.5049822360277176, 0.5224301785230636, 0.5063945114612579, 0.4465250238776207, 0.4903402656316757, 0.47245544791221616, 0.5010672137141228, 0.4827308863401413, 0.4740488171577454, 0.5244488134980202, 0.45220896881073713, 0.5562389403581619, 0.521636962890625, 0.457094606757164, 0.5193618267774582, 0.5016861736774445, 0.4560350768268108, 0.46627736687660215, 0.4643412709236145, 0.493198736384511, 0.482271334528923, 0.46111065447330474, 0.45161724388599395, 0.44265101850032806, 0.4948916271328926, 0.5291534811258316, 0.4440324455499649, 0.4889973565936089, 0.41588062793016434, 0.47836375534534453, 0.5032320886850357, 0.4595733344554901, 0.5382910311222077, 0.47193415015935897, 0.4815657526254654, 0.5348378270864487, 0.4591535121202469, 0.5314137399196625, 0.5330890417098999, 0.4637297362089157, 0.5074151128530502, 0.46779959350824357, 0.49547362327575684, 0.48850396275520325, 0.5656635522842407, 0.46026513576507566, 0.4740471735596657, 0.5022452294826507, 0.45514278411865233, 0.48062963485717775, 0.4553489089012146, 0.49109174162149427, 0.5028841763734817, 0.4902922213077545, 0.4059889271855354, 0.3862749129533768, 0.4889375641942024, 0.5263453543186187, 0.4755681544542313, 0.48354376554489137, 0.47350031286478045]

num_topics = 62

Average: 0.486547494775826 Per Topic: [0.46032810807228086, 0.4663463726639748, 0.47478739321231844, 0.46000693887472155, 0.49963329881429674, 0.514328607916832, 0.4525750517845154, 0.41100092977285385, 0.4541796326637268, 0.544234898686409, 0.45627804920077325, 0.5495024204254151, 0.5037932798266411, 0.42319113314151763, 0.47135191708803176, 0.4837930709123611, 0.47893101871013644, 0.5210075795650482, 0.47514451295137405, 0.49335914850234985, 0.49708870649337766, 0.5497853368520736, 0.5062040567398072, 0.45183271914720535, 0.5546993434429168, 0.5339938819408416, 0.4291553720831871, 0.4956794291734695, 0.46958894431591036, 0.4206779569387436, 0.5128017365932465, 0.515803050994873, 0.4660381689667702, 0.4521655380725861, 0.538817635178566, 0.5074530005455017, 0.4928511381149292, 0.450295552611351, 0.5742780119180679, 0.493124171346426, 0.5160110265016555, 0.5048722565174103, 0.4104688227176666, 0.5404113620519638, 0.502901914715767, 0.4979332685470581, 0.5253615051507949, 0.47887544333934784, 0.5632960259914398, 0.5356112480163574, 0.49976917207241056, 0.46464583873748777, 0.4781781077384949, 0.45975151658058167, 0.5404957264661789, 0.4415349394083023, 0.44373478889465334, 0.4905125260353088, 0.48433036357164383, 0.4496543526649475, 0.4206687718629837, 0.41081858426332474]

num_topics = 64

Average: 0.485499968160002 Per Topic: [0.4044230177998543, 0.47127504646778107, 0.4315623611211777, 0.42140165641903876, 0.5170104831457139, 0.5588166564702988, 0.38198396638035775, 0.4385757461190224, 0.5178005456924438, 0.4477938860654831, 0.47305606603622435, 0.5104927510023117, 0.529432912170887, 0.5065232247114182, 0.46556046009063723, 0.45559830218553543, 0.4624957233667374, 0.42726569026708605, 0.5125712305307388, 0.5151006072759629, 0.4910727560520172, 0.522339153289795, 0.4906630039215088, 0.48188421577215196, 0.4461941499263048, 0.49638532996177676, 0.4813838988542557, 0.5519891768693924, 0.4990603983402252, 0.4753400534391403, 0.42106651365756986, 0.4262001395225525, 0.5315131634473801, 0.4749322608113289, 0.5086693704128266, 0.48410085337236525, 0.5433420300483703, 0.46454509645700454, 0.49502706825733184, 0.48885852843523026, 0.49842170178890227, 0.5263402134180069, 0.5307172134518623, 0.5407624185085297, 0.5011986553668976, 0.4655129000544548, 0.45584052205085757, 0.49556874483823776, 0.4944712191820145, 0.4880408138036728, 0.5081266358494758, 0.469392229616642, 0.4981116861104965, 0.5052702128887177, 0.4726332247257233, 0.4866366103291512, 0.5430650323629379, 0.4735753446817398, 0.42833551466465, 0.4685666263103485, 0.4707404553890228, 0.4661747634410858, 0.5366178035736084, 0.5245699256658554]

num_topics = 66

Average: 0.4785651889827216 Per Topic: [0.46930366903543475, 0.45942544788122175, 0.4326218582689762, 0.512398362159729, 0.49111864268779754, 0.42731620371341705, 0.4835818290710449, 0.5070864051580429, 0.5039082676172256, 0.4569352775812149, 0.450287526845932, 0.4819337993860245, 0.459553961455822, 0.5304975479841232, 0.5140514940023422, 0.39470720328390596, 0.4662621974945068, 0.5473646432161331, 0.5880510061979294, 0.4528523325920105, 0.4924583241343498, 0.4129394836723804, 0.4841507837176323, 0.457816968858242, 0.5059221431612968, 0.4296650990843773, 0.5095359951257705, 0.5346145480871201, 0.487962144613266, 0.46633630692958833, 0.479403991997242, 0.44201753437519076, 0.4008177489042282, 0.44215416014194486, 0.4584277540445328, 0.48537069112062453, 0.5521512448787689, 0.49309519231319426, 0.4601002722978592, 0.4549419224262238, 0.4722679823637009, 0.4694948196411133, 0.5161974057555199, 0.48171475268900393, 0.47582819014787675, 0.526528611779213, 0.4264302171766758, 0.4830340802669525, 0.4937806338071823, 0.49087990671396253, 0.5080241382122039, 0.47921122461557386, 0.5116830706596375, 0.5211794197559356, 0.4203986138105392, 0.4821771293878555, 0.4082943595945835, 0.4575291432440281, 0.48250599652528764, 0.4600603073835373, 0.5025531150400638, 0.4463046044111252, 0.4650731489062309, 0.514250111579895, 0.5256904035806655, 0.48507110029459]

num_topics = 68

Average: 0.4809990682502222 Per Topic: [0.5222259178757668, 0.4776560515165329, 0.4938561484217644, 0.48508565723896024, 0.4505108743906021, 0.5100179672241211, 0.5004633158445358, 0.5059668004512787, 0.4444691836833954, 0.4875872477889061, 0.4134647071361542, 0.5135822534561157, 0.41408825311809777, 0.48136820197105407, 0.36940352618694305, 0.4374456152319908, 0.46429225653409956, 0.4860961362719536, 0.4381699874997139, 0.43185496255755423, 0.38167079389095304, 0.5155164271593093, 0.5243402868509293, 0.4648632988333702, 0.5088687151670456, 0.472376337647438, 0.42715918719768525, 0.48630448877811433, 0.4864609569311142, 0.4704268753528595, 0.5728339821100235, 0.46438525021076205, 0.4840086318552494, 0.47710897773504257, 0.5962918370962143, 0.46271636337041855, 0.4886184632778168, 0.4125222399830818, 0.4907255291938782, 0.4491922676563263, 0.5210075795650482, 0.5188352942466736, 0.419826278090477, 0.49081555753946304, 0.5084962606430053, 0.4728536993265152, 0.48519969880580904, 0.5142510056495666, 0.4904218643903732, 0.5405193209648133, 0.5102972865104676, 0.4372738182544708, 0.49962699189782145, 0.5045999616384507, 0.44783075898885727, 0.5154250800609589, 0.4288931116461754, 0.4942723661661148, 0.6485377758741379, 0.4333208084106445, 0.54489805996418, 0.4210597135126591, 0.45518704652786257, 0.4803166836500168, 0.4929358303546906, 0.4290701299905777, 0.5242402791976929, 0.5139284044504165]

num_topics = 61

Average: 0.4804229424762556

Per Topic: [0.5450899779796601, 0.47144379615783694, 0.40156030282378197, 0.47983298897743226, 0.5083923250436783, 0.5098015680909157, 0.3890345752239227, 0.45928924046456815, 0.5104124993085861, 0.47595606818795205, 0.45721154361963273, 0.4234661117196083, 0.49284549951553347, 0.46317347288131716, 0.5745804846286774, 0.4533018499612808, 0.45115862786769867, 0.5515768080949783, 0.4858097642660141, 0.45243210047483445, 0.4644439786672592, 0.4687983587384224, 0.4767948120832443, 0.4725842255167663, 0.46516810059547425, 0.4893561273813248, 0.47340834736824033, 0.445457486435771, 0.4805245965719223, 0.5510461628437042, 0.5291787564754487, 0.49029498975723984, 0.49846934974193574, 0.5203188106417656, 0.42381062284111976, 0.5442034065723419, 0.46878657042980193, 0.47264925539493563, 0.4698571145534515, 0.4666514337062836, 0.47120439559221267, 0.4552793025970459, 0.4035217121243477, 0.4990536034107208, 0.44353742003440855, 0.4447066769003868, 0.5409434825181961, 0.5478465586900712, 0.5138252288103103, 0.4609436303377151, 0.48352972567081454, 0.5002520114183426, 0.5305933684110642, 0.45883954390883447, 0.44574637711048126, 0.4418002188205719, 0.48118861615657804, 0.5552921324968338, 0.4547287732362747, 0.4237870216369629, 0.5210075795650482]

num_topics = 63

Average: 0.4919715766987157

Per Topic: [0.4972303718328476, 0.42189271152019503, 0.4716315895318985, 0.48867475986480713, 0.5294153422117234, 0.5184957563877106, 0.4488393545150757, 0.5155438363552094, 0.4954787611961365, 0.498890370875597, 0.4401729732751846, 0.5177865147590637, 0.4709128186106682, 0.5513818442821503, 0.5421865195035934, 0.45075089335441587, 0.5109215199947357, 0.410452227294445, 0.472388955950737, 0.45020095109939573, 0.4676086835563183, 0.45408256649971007, 0.5168567150831223, 0.5600411891937256, 0.49693180322647096, 0.49827553033828736, 0.4560760110616684, 0.5661256566643715, 0.4280627012252808, 0.48303947150707244, 0.445560222864151, 0.4220785770565271, 0.48177587389945986, 0.5084250658750534, 0.5127225816249847, 0.5034349203109741, 0.5174091815948486, 0.4951315581798553, 0.4640683338046074, 0.5027432158589363, 0.4189660683274269, 0.5010650455951691, 0.48148939460515977, 0.5037121295928955, 0.5087469011545181, 0.49310308396816255, 0.4951219201087952, 0.4853700459003448, 0.5309717237949372, 0.5113401651382447, 0.5107899606227875, 0.4865313768386841, 0.5195753782987594, 0.49279720783233644, 0.5216559380292892, 0.5623087372630835, 0.49507503509521483, 0.4869211450219154, 0.5389730304479599, 0.4980350524187088, 0.566211485862732, 0.4276759184896946, 0.4740746557712555]

In [ ]:

# 定义205个全局主题
global_topics_205 = [
    "AI rights", "digital clone of a person", "speculative virtual reality", "augmented reality", "coping with being tortured",
    "speculative punishment", "what if I killed someone by accident", "compromising one's ethical principles vs. self-preservation",
    "facing a hostage situation", "saving innocent lives vs. compromising one's ethical principles", "bestiality",
    "human self-sacrifice for another", "husband and wife", "social media in society", "celebrity culture in society",
    "consumerist dystopia", "escape from reality", "mass consumerism", "pervasive marketing in society", "prostitution",
    "prostitution vs. Poverty", "selling out for money", "the desire for fame", "the need for a purpose in life", "unrequited love",
    "coping with a jealous lover", "recording implant", "romantic infidelity", "romantic jealousy", "brain-computer interface",
    "invasion of privacy", "mass surveillance", "obsession", "old flames", "romantic love", "security vs. Freedom",
    "android clone of a person", "coping with the death of a family member", "duplicate of a person", "engaged couple",
    "personal identity", "crime and punishment", "cruel and unusual punishment", "eye for an eye justice", "prisoners being exploited for entertainment",
    "treatment of prisoners", "lawless anarchy dystopia", "murder", "what if everyone turned into zombies", "what if I had amnesia",
    "what if I woke up in a strange place and didn't remember how I got there", "coping with personal failure", "democracy",
    "duty vs. conscience", "infatuation", "political apathy in society", "professional politics", "the desire for professional success",
    "conformism in society", "peer pressure", "social media dystopia", "cracking under pressure", "descent into crime", "fall from grace",
    "friendship", "the desire to be liked", "facing one's darkest fear", "fear", "mother and son", "the nature of fear",
    "the nature of the subconscious", "coping with a family issue", "coping with the death of a parent", "loyalty to one's family",
    "remorse", "what if I was trapped inside a game", "pedophiles", "blackmail", "Internet crime in society", "vigilante justice",
    "digital afterlife", "coming to terms with one's own death", "coping with a disability", "coping with a loved one being gravely ill",
    "coping with a terminal illness", "coping with aging", "extrovert vs. Introvert", "girlfriend and girlfriend", "love vs. Faith",
    "old-age love", "old-age romance", "dehumanization", "military related work", "supersoldier", "destroying one people to save another",
    "ethnic hatred", "genocide", "humanoid mutant", "military indoctrination", "mechanical insect", "out of control AI",
    "society regulating central computer crash", "endangered species", "taking the law into one's own hands", "young character vs. old character",
    "virtual person", "human vs. Captivity", "psychopathy", "self-confidence", "the universe is a simulation", "human parenting",
    "monitoring implant", "protecting children vs. letting them learn", "the road to hell is paved with good intentions", "coping with a meddling parent",
    "mother and daughter", "parental controls in society", "single motherhood", "having a skeleton in the closet", "mind reading technology",
    "forbidden love", "looking for a lover", "matchmaking technology", "romantic relationship", "love at first sight", "machine controlled society",
    "romantic courtship", "autonomous weapons system killing everyone", "facing organized pursuit", "human vs. sentient machine",
    "what if I were all alone in the world", "mind linking technology", "mind transfer technology", "addiction to sensation", "controlling partner",
    "experimental medical procedure", "racism in society", "sadomasochism", "the desire for vengeance", "unethical human experimentation",
    "what if I were in a coma-like state", "love triangle", "matrimonial love", "coming to terms with one's own sexuality", "gender bending",
    "human addiction", "love vs. Friendship", "male homosexuality", "sexual norms in society", "video game addiction", "virtual reality sex",
    "coping with suicide", "social media addiction", "suicide", "beating one's self up", "coping with the death of a child", "coping with the death of a lover",
    "social inequality", "the business world", "the need for closure", "greed for riches", "AI companion", "aunt and niece", "father and daughter",
    "loneliness", "mind uploading", "music", "role models", "single fatherhood", "sister and sister", "stepmother and stepdaughter", "teenage angst",
    "the need for companionship", "onerous user agreements", "boss and employee", "coping with humiliation", "nested universe", "personality rights",
    "romantic relationship issue", "the need for privacy", "what if my life were different", "serial murder", "appearances can be deceiving",
    "filmmaking", "boyfriend and girlfriend", "Mind transfer technology", "coping with the death of one's entire family", "falling out of friendship",
    "misery loves company", "space travel", "speculative robotics", "speculative spaceship", "compromising one's ethical principles for personal advantage",
    "werewolf", "compromising one's ethical principles vs. great need", "human vs. Monster", "overcoming an addiction", "what if the fate of the world was in my hands",
    "demon", "hallucination of a non-existing person", "human sacrifice", "killing to survive", "nuclear holocaust", "the religious end of the world",
    "workplace rivalry"
]

!pip install scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity # Import cosine_similarity
import numpy as np # Import the NumPy library

# Convert topics to vector representations using TF-IDF
vectorizer = TfidfVectorizer()
all_topics = topics + global_topics_205
vectorized_topics = vectorizer.fit_transform(all_topics)

# Split the vectorized topics into two sets
vec_63 = vectorized_topics[:len(topics)]
vec_205 = vectorized_topics[len(topics):]

# Compute cosine similarity between the 63 topics and the 205 global topics
similarity_matrix = cosine_similarity(vec_63, vec_205)

# For each of the 63 topics, find the most similar global topics
top_n = 5  # Top n global topics to consider for each topic
coherence_scores = []
for i, row in enumerate(similarity_matrix):
    top_similarities = np.sort(row)[-top_n:]
    avg_similarity = np.mean(top_similarities)
    coherence_scores.append(avg_similarity)
    print(f"Topic {i} Top {top_n} Matches with Global Topics:")
    top_indices = np.argsort(row)[-top_n:]
    for idx in top_indices:
        print(f"Global Topic {idx}: Similarity = {row[idx]}")

# Overall coherence score
overall_coherence = np.mean(coherence_scores)
print(f"Overall Coherence Score: {overall_coherence}")


Topic 0 Top 5 Matches with Global Topics:
Global Topic 71: Similarity = 0.0
Global Topic 72: Similarity = 0.0
Global Topic 73: Similarity = 0.0
Global Topic 63: Similarity = 0.0
Global Topic 204: Similarity = 0.0
Topic 1 Top 5 Matches with Global Topics:
Global Topic 71: Similarity = 0.0
Global Topic 72: Similarity = 0.0
Global Topic 73: Similarity = 0.0
Global Topic 63: Similarity = 0.0
Global Topic 204: Similarity = 0.0
Topic 2 Top 5 Matches with Global Topics:
Global Topic 71: Similarity = 0.0
Global Topic 72: Similarity = 0.0
Global Topic 73: Similarity = 0.0
Global Topic 63: Similarity = 0.0
Global Topic 204: Similarity = 0.0
Topic 3 Top 5 Matches with Global Topics:
Global Topic 63: Similarity = 0.0
Global Topic 104: Similarity = 0.08664857705274373
Global Topic 90: Similarity = 0.13517044943349493
Global Topic 89: Similarity = 0.15012864822078764
Global Topic 33: Similarity = 0.1674809961132203
Topic 4 Top 5 Matches with Global Topics:
Global Topic 73: Similarity = 0.0
Global To

In [ ]:
# prompt: list top 10 topics out of 63 that matches the global topics most according to the results above，with their  Top 5 Matches with Global Topics similarity，show the full name and number of  the global topic

import numpy as np
# Sort the coherence scores and get the indices of the top 10 topics
top_10_indices = np.argsort(coherence_scores)[-10:]

print("Top 10 Topics with Highest Coherence with Global Topics:")
for i, topic_idx in enumerate(top_10_indices):
    print(f"\nRank {i+1}: Topic {topic_idx} ({topics[topic_idx]}) - Coherence Score: {coherence_scores[topic_idx]}")
    top_global_indices = np.argsort(similarity_matrix[topic_idx])[-top_n:]
    for global_idx in top_global_indices:
        print(f"  - Global Topic {global_idx} ({global_topics_205[global_idx]}): Similarity = {similarity_matrix[topic_idx][global_idx]}")


Top 10 Topics with Highest Coherence with Global Topics:

Rank 1: Topic 51 (police never person call playing finally people things admits drives) - Coherence Score: 0.13281199491463386
  - Global Topic 36 (android clone of a person): Similarity = 0.11462701067038866
  - Global Topic 1 (digital clone of a person): Similarity = 0.12100574096452903
  - Global Topic 94 (destroying one people to save another): Similarity = 0.12291946700134439
  - Global Topic 38 (duplicate of a person): Similarity = 0.1380784276509244
  - Global Topic 105 (virtual person): Similarity = 0.16742932828598284

Rank 2: Topic 39 (back picture symbol found knife hands door people person wall) - Coherence Score: 0.1328739596383007
  - Global Topic 103 (taking the law into one's own hands): Similarity = 0.11784559097683611
  - Global Topic 1 (digital clone of a person): Similarity = 0.12036512367096426
  - Global Topic 94 (destroying one people to save another): Similarity = 0.12226871823811086
  - Global Topic 38 (